In [1]:
import dxpy
import dxdata
import pandas as pd
import pyspark
import subprocess
from pyspark.sql import SQLContext
from pyspark import SparkConf, SparkContext

dxdata.__version__
connection = dxdata.connect()
conf = pyspark.SparkConf().set("spark.kryoserializer.buffer.max", "2000m")

/opt/conda/lib/python3.11/site-packages/thrift/transport/TSSLSocket.py:53: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  self._context = ssl.SSLContext(ssl_version)


In [2]:
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)

In [3]:
# Automatically discover dispensed database name and dataset id
dispensed_database = dxpy.find_one_data_object(
    classname='database', 
    name='app*', 
    folder='/', 
    name_mode='glob', 
    describe=True)
dispensed_database_name = dispensed_database['describe']['name']

dispensed_dataset = dxpy.find_one_data_object(
    typename='Dataset', 
    name='app*.dataset', 
    folder='/', 
    name_mode='glob')
dispensed_dataset_id = dispensed_dataset['id']
dataset = dxdata.load_dataset(id=dispensed_dataset_id)
dataset.entities

[<Entity "participant">,
 <Entity "covid19_result_england">,
 <Entity "covid19_result_scotland">,
 <Entity "covid19_result_wales">,
 <Entity "gp_clinical">,
 <Entity "gp_scripts">,
 <Entity "gp_registrations">,
 <Entity "hesin">,
 <Entity "hesin_diag">,
 <Entity "hesin_oper">,
 <Entity "hesin_critical">,
 <Entity "hesin_maternity">,
 <Entity "hesin_delivery">,
 <Entity "hesin_psych">,
 <Entity "death">,
 <Entity "death_cause">,
 <Entity "olink_instance_0">,
 <Entity "olink_instance_2">,
 <Entity "olink_instance_3">]

In [4]:
participant = dataset['participant']

In [5]:
# Returns all field objects for a given UKB showcase field id
def fields_for_id(field_id):
    from distutils.version import LooseVersion
    field_id = str(field_id)
    fields = participant.find_fields(name_regex=r'^p{}(_i\d+)?(_a\d+)?$'.format(field_id))
    return sorted(fields, key=lambda f: LooseVersion(f.name))

# Returns all field names for a given UKB showcase field id
def field_names_for_id(field_id):
    return [f.name for f in fields_for_id(field_id)]

In [ ]:
# split the recommended fields into a list
merged_fields = pd.read_csv("merged_fields.tsv", sep="\t", header=0)
merge_field_list = merged_fields["field_id"].tolist()
# cut the recommended fields list for 20 fields each group
merge_field_list = [merge_field_list[i:i + 20] for i in range(0, len(merge_field_list), 20)]
for i, group in enumerate(merge_field_list):
    if i <= 13:
        continue
    print(f"Retrieving fields for group {i + 1} with {len(group)} fields")
    # change the group astype to string
    group = [str(x) for x in group]
    get_ids = sum([field_names_for_id(field_id) for field_id in group], [])
    field_names = ['eid'] + get_ids
    df = participant.retrieve_fields(names=field_names, engine=connection)
    data = df.toPandas()
    display(data)
    # save the data to a file
    # if the file already exists, continue to the next iteration
    data.to_csv(f'./core_category/fields_group_{i + 1}.tsv', sep='\t', index=False, header=True)
    # Upload the file using subprocess
    subprocess.run([
        "dx", "upload", f"./core_category/fields_group_{i + 1}.tsv",
        "-p", "--path", "/Output/Traits/core_category/", "--brief"
    ], check=True)

Retrieving fields for group 15 with 20 fields


,eid,p3160_i0,p3160_i1,p3160_i2,p3160_i3,p3166_i0_a0,p3166_i0_a1,p3166_i0_a2,p3166_i0_a3,p3166_i0_a4,...,p3571_i2,p3571_i3,p3581_i0,p3581_i1,p3581_i2,p3581_i3,p3591_i0,p3591_i1,p3591_i2,p3591_i3
0,1000122,NaN,NaN,NaN,NaN,2009-09-15 09:37:24,2009-09-15 09:37:26,2009-09-15 09:37:28,2009-09-15 09:37:31,2009-09-15 09:37:34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000150,NaN,NaN,NaN,NaN,2007-08-25 13:04:52,2007-08-25 13:04:49,2007-08-25 13:04:46,2007-08-25 13:04:56,2007-08-25 13:05:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000163,NaN,NaN,NaN,NaN,2008-09-09 17:33:43,2008-09-09 17:33:39,2008-09-09 17:33:47,2008-09-09 17:33:50,2008-09-09 17:33:52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000196,NaN,NaN,NaN,NaN,2010-01-28 18:19:27,2010-01-28 18:19:31,2010-01-28 18:19:21,2010-01-28 18:19:27,2010-01-28 18:19:24,...,NaN,NaN,45.0,NaN,51.0,NaN,0.0,NaN,0.0,NaN
4,1000221,NaN,NaN,NaN,NaN,2009-07-20 09:42:16,2009-07-20 09:42:20,2009-07-20 09:42:23,2009-07-20 09:42:26,2009-07-20 09:42:30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502123,6021183,NaN,NaN,NaN,NaN,2009-06-03 17:52:36,2009-06-03 17:52:39,2009-06-03 17:52:42,2009-06-03 17:52:44,2009-06-03 17:52:46,...,NaN,NaN,49.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN
502124,6021216,NaN,NaN,NaN,NaN,2008-05-29 16:48:54,2008-05-29 16:49:00,2008-05-29 16:49:11,2008-05-29 16:49:24,2008-05-29 16:49:32,...,1.0,NaN,NaN,NaN,54.0,NaN,0.0,NaN,0.0,NaN
502125,6021303,NaN,NaN,NaN,NaN,2008-04-08 11:48:38,2008-04-08 11:48:41,2008-04-08 11:48:37,2008-04-08 11:48:42,2008-04-08 11:48:40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502126,6021358,NaN,NaN,NaN,NaN,2008-09-24 11:17:11,2008-09-24 11:17:13,2008-09-24 11:17:15,2008-09-24 11:17:18,2008-09-24 11:17:20,...,NaN,NaN,54.0,NaN,58.0,NaN,0.0,NaN,0.0,NaN


Retrieving fields for group 16 with 20 fields


,eid,p3606_i0,p3606_i1,p3606_i2,p3606_i3,p3616_i0,p3616_i1,p3616_i2,p3616_i3,p3627_i0,...,p3799_i2,p3799_i3,p3809_i0,p3809_i1,p3809_i2,p3809_i3,p3829_i0,p3829_i1,p3829_i2,p3829_i3
0,1000122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,-10.0,NaN,NaN,NaN,NaN,NaN
1,1000150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,12.0,NaN,20.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502123,6021183,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
502124,6021216,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502125,6021303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502126,6021358,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Retrieving fields for group 17 with 20 fields


...

Retrieving fields for group 18 with 20 fields


,eid,p4093_i0,p4093_i1,p4093_i2,p4093_i3,p4095_i0,p4095_i1,p4095_i2,p4095_i3,p4096_i0,...,p4139_i2,p4139_i3,p4140_i0,p4140_i1,p4140_i2,p4140_i3,p4141_i0,p4141_i1,p4141_i2,p4141_i3
0,1451876,NaN,0.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3046819,0.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5015491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1598419,0.0,NaN,NaN,NaN,1.0,NaN,7.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3293483,0.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502123,2998069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502124,3834454,0.0,NaN,0.0,NaN,1.0,NaN,1.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502125,5758738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502126,5115563,0.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Retrieving fields for group 19 with 20 fields


,eid,p4142_i0,p4142_i1,p4142_i2,p4142_i3,p4143_i0,p4143_i1,p4143_i2,p4143_i3,p4144_i0,...,p4232_i3_a6,p4232_i3_a7,p4232_i3_a8,p4232_i3_a9,p4232_i3_a10,p4232_i3_a11,p4232_i3_a12,p4232_i3_a13,p4232_i3_a14,p4232_i3_a15
0,1000122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502123,6021183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502124,6021216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502125,6021303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502126,6021358,NaN,NaN,1542.2,NaN,NaN,NaN,-0.3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Retrieving fields for group 20 with 20 fields
